In [9]:
from lowe.acs.ACSClient import ACSClient
from lowe.locations.lookup import name2fips, fips2name, search
import pandas as pd
import plotly.graph_objects as go

In [10]:
client = ACSClient()
await client.initialize()

In [11]:
CATHEDRAL_CITY = "0612048"
COACHELLA = "0614260"
DESERT_HS = "0618996"
INDIAN_WELLS = "0636434"
INDIO = "0636448"
LA_QUINTA = "0640354"
PALM_DESERT = "0655184"
PALM_SPRINGS = "0655254"
RANCHO_MIRAGE = "0659500"

cities = [CATHEDRAL_CITY, COACHELLA, DESERT_HS, INDIAN_WELLS, INDIO, LA_QUINTA, PALM_DESERT, PALM_SPRINGS, RANCHO_MIRAGE]

locs = [{"city": cityname} for cityname in cities]


current_year = "2016"

In [12]:
resp = await client.get_acs(
    vars=["S0101"],
    start_year="2012",
    end_year="2019",
    location=locs,
    estimate="5",
    debug=False)

In [13]:
cols_mask = resp.columns.str.contains("AGE AND SEX Estimate Percent|city|location_key")
resp = resp.loc[:, cols_mask]
no_gender_mask = ~resp.columns.str.contains("male|female|Male|Female|MALE|FEMALE")
resp = resp.loc[:, no_gender_mask]
other_mask = ~resp.columns.str.contains("SELECTED AGE CATEGORIES|SUMMARY INDICATORS|PERCENT ALLOCATED")
resp = resp.loc[:, other_mask]

In [14]:
for col in resp.columns:
    print(col)

AGE AND SEX Estimate Percent Total population
AGE AND SEX Estimate Percent Total population AGE 10 to 14 years
AGE AND SEX Estimate Percent Total population AGE 15 to 19 years
AGE AND SEX Estimate Percent Total population AGE 20 to 24 years
AGE AND SEX Estimate Percent Total population AGE 25 to 29 years
AGE AND SEX Estimate Percent Total population AGE 30 to 34 years
AGE AND SEX Estimate Percent Total population AGE 35 to 39 years
AGE AND SEX Estimate Percent Total population AGE 40 to 44 years
AGE AND SEX Estimate Percent Total population AGE 45 to 49 years
AGE AND SEX Estimate Percent Total population AGE 5 to 9 years
AGE AND SEX Estimate Percent Total population AGE 50 to 54 years
AGE AND SEX Estimate Percent Total population AGE 55 to 59 years
AGE AND SEX Estimate Percent Total population AGE 60 to 64 years
AGE AND SEX Estimate Percent Total population AGE 65 to 69 years
AGE AND SEX Estimate Percent Total population AGE 70 to 74 years
AGE AND SEX Estimate Percent Total population 

In [15]:
newcols = []
for col in resp.columns:
    newcols.append(col.replace("AGE AND SEX Estimate Percent Total population AGE ", ""))
resp.columns = newcols

In [16]:
resp["0-14 years perc"] = resp["Under 5 years"].astype(float) + resp["5 to 9 years"].astype(float) + resp["10 to 14 years"].astype(float)
resp["15-24 years perc"] = resp["15 to 19 years"].astype(float) + resp["20 to 24 years"].astype(float)
resp["25-34 years perc"] = resp["25 to 29 years"].astype(float) + resp["30 to 34 years"].astype(float)
resp["35-44 years perc"] = resp["35 to 39 years"].astype(float) + resp["40 to 44 years"].astype(float)
resp["45-64 years perc"] = resp["45 to 49 years"].astype(float) + resp["50 to 54 years"].astype(float) + resp["55 to 59 years"].astype(float) + resp["60 to 64 years"].astype(float)
resp["65+ years perc"] = resp["65 to 69 years"].astype(float) + resp["70 to 74 years"].astype(float) + resp["75 to 79 years"].astype(float) + resp["80 to 84 years"].astype(float) + resp["85 years and over"].astype(float)


resp = resp.rename(columns = {"AGE AND SEX Estimate Percent Total population": "Total population"})

resp["0-14 years"] = resp["0-14 years perc"].astype(float) * resp["Total population"].astype(float) * 0.01
resp["15-24 years"] = resp["15-24 years perc"].astype(float) * resp["Total population"].astype(float) * 0.01
resp["25-34 years"] = resp["25-34 years perc"].astype(float) * resp["Total population"].astype(float) * 0.01
resp["35-44 years"] = resp["35-44 years perc"].astype(float) * resp["Total population"].astype(float) * 0.01
resp["45-64 years"] = resp["45-64 years perc"].astype(float) * resp["Total population"].astype(float) * 0.01
resp["65+ years"] = resp["65+ years perc"].astype(float) * resp["Total population"].astype(float) * 0.01


In [17]:
cols = ["location_key", "0-14 years perc", "15-24 years perc", "25-34 years perc",
             "35-44 years perc","45-64 years perc","65+ years perc",
             "0-14 years", "15-24 years", "25-34 years", 
             "35-44 years","45-64 years","65+ years", "Total population"]

resp = resp[cols]

resp["Total population"] = resp["Total population"].astype(float)

In [18]:
cv = resp.groupby(["year"]).sum()
cv["0-14 years perc"] = cv["0-14 years"] / cv["Total population"]
cv["15-24 years perc"] = cv["15-24 years"] / cv["Total population"]
cv["25-34 years perc"] = cv["25-34 years"] / cv["Total population"]
cv["35-44 years perc"] = cv["35-44 years"] / cv["Total population"]
cv["45-64 years perc"] = cv["45-64 years"] / cv["Total population"]
cv["65+ years perc"] = cv["65+ years"] / cv["Total population"]
cv["location_key"] = "Coachella Valley"

cv = cv[cols]

In [19]:
cv

together = pd.concat([resp, cv])
together

,location_key,0-14 years perc,15-24 years perc,25-34 years perc,35-44 years perc,45-64 years perc,65+ years perc,0-14 years,15-24 years,25-34 years,35-44 years,45-64 years,65+ years,Total population
year,,,,,,,,,,,,,,
2012,cathedral city ca,20.700000,16.600000,11.500000,12.600000,24.700000,13.700000,5.312034e+03,4.259892e+03,2.951130e+03,3.233412e+03,6.338514e+03,3.515694e+03,2.566200e+04
2012,coachella ca,34.500000,16.300000,17.300000,12.600000,15.300000,4.000000,7.110105e+03,3.359267e+03,3.565357e+03,2.596734e+03,3.153177e+03,8.243600e+02,2.060900e+04
2012,desert hot springs ca,28.400000,16.700000,13.800000,10.700000,20.900000,9.600000,3.793672e+03,2.230786e+03,1.843404e+03,1.429306e+03,2.791822e+03,1.282368e+03,1.335800e+04
2012,indian wells ca,3.300000,6.700000,1.500000,1.300000,31.400000,55.800000,7.702200e+01,1.563780e+02,3.501000e+01,3.034200e+01,7.328760e+02,1.302372e+03,2.334000e+03
2012,indio ca,26.200000,14.500000,14.700000,12.700000,18.800000,13.100000,9.751378e+03,5.396755e+03,5.471193e+03,4.726813e+03,6.997172e+03,4.875689e+03,3.721900e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,Coachella Valley,0.197702,0.128349,0.120468,0.112190,0.245926,0.194722,3.692001e+04,2.396870e+04,2.249687e+04,2.095101e+04,4.592562e+04,3.636348e+04,1.867460e+05
2016,Coachella Valley,0.191105,0.121430,0.120182,0.112948,0.249604,0.204792,3.602315e+04,2.288949e+04,2.265410e+04,2.129051e+04,4.705006e+04,3.860311e+04,1.884990e+05
2017,Coachella Valley,0.158111,0.098667,0.101889,0.105778,0.259556,0.276111,-1.264889e+09,-7.893333e+08,-8.151111e+08,-8.462222e+08,-2.076444e+09,-2.208889e+09,-8.000000e+09


In [20]:
resp["Total population"]

year
2012        25662.0
2012        20609.0
2012        13358.0
2012         2334.0
2012        37219.0
           ...     
2019   -888888888.0
2019   -888888888.0
2019   -888888888.0
2019   -888888888.0
2019   -888888888.0
Name: Total population, Length: 72, dtype: float64

In [21]:
[together[location_key].unique() for location_key in together.columns]

[array(['cathedral city ca', 'coachella ca', 'desert hot springs ca',
        'indian wells ca', 'indio ca', 'la quinta ca', 'palm desert ca',
        'palm springs ca', 'rancho mirage ca', 'Coachella Valley'],
       dtype=object),
 array([20.7       , 34.5       , 28.4       ,  3.3       , 26.2       ,
        19.1       , 13.6       ,  8.7       ,  7.        , 20.4       ,
        34.7       , 27.9       ,  2.5       , 25.7       , 19.5       ,
        13.1       ,  9.4       ,  8.3       , 21.5       , 32.5       ,
        28.3       ,  1.9       , 25.6       , 17.2       , 14.6       ,
         8.2       , 21.        , 31.5       , 28.7       , 24.9       ,
        17.        , 13.2       ,  9.2       ,  8.9       , 29.2       ,
        27.9       ,  3.        , 24.1       , 16.3       , 13.9       ,
         9.1       ,  7.2       , 20.2       , 25.        ,  3.9       ,
        21.9       , 16.5       , 13.4       , 10.1       ,  6.4       ,
        19.7       , 21.2       , 25.

In [22]:
rc_code = {"county": "06_065"}

In [23]:
rc = await client.get_acs(
    vars=["S0101"],
    start_year="2012",
    end_year="2019",
    location=rc_code,
    estimate="5",
    debug=False)

In [24]:
cols_mask = rc.columns.str.contains("AGE AND SEX Estimate Percent|city|location_key")
rc = rc.loc[:, cols_mask]
no_gender_mask = ~rc.columns.str.contains("male|female|Male|Female|MALE|FEMALE")
rc = rc.loc[:, no_gender_mask]
other_mask = ~rc.columns.str.contains("SELECTED AGE CATEGORIES|SUMMARY INDICATORS|PERCENT ALLOCATED")
rc = rc.loc[:, other_mask]

In [25]:
for col in rc.columns:
    print(col)

AGE AND SEX Estimate Percent Total population
AGE AND SEX Estimate Percent Total population AGE 10 to 14 years
AGE AND SEX Estimate Percent Total population AGE 15 to 19 years
AGE AND SEX Estimate Percent Total population AGE 20 to 24 years
AGE AND SEX Estimate Percent Total population AGE 25 to 29 years
AGE AND SEX Estimate Percent Total population AGE 30 to 34 years
AGE AND SEX Estimate Percent Total population AGE 35 to 39 years
AGE AND SEX Estimate Percent Total population AGE 40 to 44 years
AGE AND SEX Estimate Percent Total population AGE 45 to 49 years
AGE AND SEX Estimate Percent Total population AGE 5 to 9 years
AGE AND SEX Estimate Percent Total population AGE 50 to 54 years
AGE AND SEX Estimate Percent Total population AGE 55 to 59 years
AGE AND SEX Estimate Percent Total population AGE 60 to 64 years
AGE AND SEX Estimate Percent Total population AGE 65 to 69 years
AGE AND SEX Estimate Percent Total population AGE 70 to 74 years
AGE AND SEX Estimate Percent Total population 

In [26]:
newcols = []
for col in rc.columns:
    newcols.append(col.replace("AGE AND SEX Estimate Percent Total population AGE ", ""))
rc.columns = newcols

In [27]:
rc

,AGE AND SEX Estimate Percent Total population,10 to 14 years,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,45 to 49 years,5 to 9 years,50 to 54 years,55 to 59 years,60 to 64 years,65 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 years and over,Under 5 years,location_key
year,,,,,,,,,,,,,,,,,,,,
2012,1091924,8.3,8.7,7.4,6.7,6.4,6.5,6.9,7.0,7.7,6.4,5.2,4.3,3.4,2.7,2.2,1.5,1.1,7.6,riverside county ca
2013,1109691,8.3,8.5,7.6,6.8,6.5,6.4,6.7,6.9,7.6,6.5,5.3,4.5,3.6,2.7,2.2,1.5,1.1,7.4,riverside county ca
2014,1127924,8.0,8.3,7.6,6.9,6.6,6.4,6.6,6.8,7.6,6.6,5.4,4.7,3.7,2.8,2.1,1.5,1.2,7.3,riverside county ca
2015,1143477,7.8,8.1,7.7,7.0,6.7,6.3,6.5,6.7,7.6,6.6,5.6,4.7,3.8,2.9,2.2,1.6,1.2,7.1,riverside county ca
2016,1156126,7.7,7.9,7.6,7.1,6.7,6.3,6.4,6.6,7.6,6.6,5.7,4.8,4.0,3.0,2.2,1.6,1.2,7.0,riverside county ca
2017,-888888888,7.5,7.6,7.3,7.0,6.7,6.4,6.4,6.4,7.2,6.4,5.8,5.1,4.4,3.3,2.5,1.7,1.6,6.7,riverside county ca
2018,-888888888,7.5,7.5,7.1,7.1,6.7,6.4,6.4,6.4,7.1,6.4,6.0,5.2,4.5,3.4,2.5,1.7,1.7,6.6,riverside county ca
2019,-888888888,7.5,7.4,7.0,7.2,6.7,6.4,6.3,6.3,7.0,6.3,6.1,5.2,4.5,3.5,2.7,1.8,1.7,6.5,riverside county ca


In [28]:
rc["0-14 years perc"] = rc["Under 5 years"].astype(float) + rc["5 to 9 years"].astype(float) + rc["10 to 14 years"].astype(float)
rc["15-24 years perc"] = rc["15 to 19 years"].astype(float) + rc["20 to 24 years"].astype(float)
rc["25-34 years perc"] = rc["25 to 29 years"].astype(float) + rc["30 to 34 years"].astype(float)
rc["35-44 years perc"] = rc["35 to 39 years"].astype(float) + rc["40 to 44 years"].astype(float)
rc["45-64 years perc"] = rc["45 to 49 years"].astype(float) + rc["50 to 54 years"].astype(float) + rc["55 to 59 years"].astype(float) + rc["60 to 64 years"].astype(float)
rc["65+ years perc"] = rc["65 to 69 years"].astype(float) + rc["70 to 74 years"].astype(float) + rc["75 to 79 years"].astype(float) + rc["80 to 84 years"].astype(float) + rc["85 years and over"].astype(float)


rc = rc.rename(columns = {"AGE AND SEX Estimate Percent Total population": "Total population"})

rc["0-14 years"] = rc["0-14 years perc"].astype(float) * rc["Total population"].astype(float) * 0.01
rc["15-24 years"] = rc["15-24 years perc"].astype(float) * rc["Total population"].astype(float) * 0.01
rc["25-34 years"] = rc["25-34 years perc"].astype(float) * rc["Total population"].astype(float) * 0.01
rc["35-44 years"] = rc["35-44 years perc"].astype(float) * rc["Total population"].astype(float) * 0.01
rc["45-64 years"] = rc["45-64 years perc"].astype(float) * rc["Total population"].astype(float) * 0.01
rc["65+ years"] = rc["65+ years perc"].astype(float) * rc["Total population"].astype(float) * 0.01


In [29]:
cols = ["location_key", "0-14 years perc", "15-24 years perc", "25-34 years perc",
             "35-44 years perc","45-64 years perc","65+ years perc",
             "0-14 years", "15-24 years", "25-34 years", 
             "35-44 years","45-64 years","65+ years", "Total population"]

rc = rc[cols]

rc["Total population"] = rc["Total population"].astype(float)

In [30]:
rc

,location_key,0-14 years perc,15-24 years perc,25-34 years perc,35-44 years perc,45-64 years perc,65+ years perc,0-14 years,15-24 years,25-34 years,35-44 years,45-64 years,65+ years,Total population
year,,,,,,,,,,,,,,
2012,riverside county ca,23.6,16.1,13.1,13.4,22.9,10.9,2.576941e+05,1.757998e+05,1.430420e+05,1.463178e+05,2.500506e+05,1.190197e+05,1091924.0
2013,riverside county ca,23.3,16.1,13.3,13.1,23.2,11.1,2.585580e+05,1.786603e+05,1.475889e+05,1.453695e+05,2.574483e+05,1.231757e+05,1109691.0
2014,riverside county ca,22.9,15.9,13.5,13.0,23.5,11.3,2.582946e+05,1.793399e+05,1.522697e+05,1.466301e+05,2.650621e+05,1.274554e+05,1127924.0
2015,riverside county ca,22.5,15.8,13.7,12.8,23.6,11.7,2.572823e+05,1.806694e+05,1.566563e+05,1.463651e+05,2.698606e+05,1.337868e+05,1143477.0
2016,riverside county ca,22.3,15.5,13.8,12.7,23.7,12.0,2.578161e+05,1.791995e+05,1.595454e+05,1.468280e+05,2.740019e+05,1.387351e+05,1156126.0
2017,riverside county ca,21.4,14.9,13.7,12.8,23.7,13.5,-1.902222e+08,-1.324444e+08,-1.217778e+08,-1.137778e+08,-2.106667e+08,-1.200000e+08,-888888888.0
2018,riverside county ca,21.2,14.6,13.8,12.8,24.0,13.8,-1.884444e+08,-1.297778e+08,-1.226667e+08,-1.137778e+08,-2.133333e+08,-1.226667e+08,-888888888.0
2019,riverside county ca,21.0,14.4,13.9,12.7,23.9,14.2,-1.866667e+08,-1.280000e+08,-1.235556e+08,-1.128889e+08,-2.124444e+08,-1.262222e+08,-888888888.0


In [31]:
final = pd.concat([together, rc])
final

,location_key,0-14 years perc,15-24 years perc,25-34 years perc,35-44 years perc,45-64 years perc,65+ years perc,0-14 years,15-24 years,25-34 years,35-44 years,45-64 years,65+ years,Total population
year,,,,,,,,,,,,,,
2012,cathedral city ca,20.7,16.6,11.5,12.6,24.7,13.7,5.312034e+03,4.259892e+03,2.951130e+03,3.233412e+03,6.338514e+03,3.515694e+03,25662.0
2012,coachella ca,34.5,16.3,17.3,12.6,15.3,4.0,7.110105e+03,3.359267e+03,3.565357e+03,2.596734e+03,3.153177e+03,8.243600e+02,20609.0
2012,desert hot springs ca,28.4,16.7,13.8,10.7,20.9,9.6,3.793672e+03,2.230786e+03,1.843404e+03,1.429306e+03,2.791822e+03,1.282368e+03,13358.0
2012,indian wells ca,3.3,6.7,1.5,1.3,31.4,55.8,7.702200e+01,1.563780e+02,3.501000e+01,3.034200e+01,7.328760e+02,1.302372e+03,2334.0
2012,indio ca,26.2,14.5,14.7,12.7,18.8,13.1,9.751378e+03,5.396755e+03,5.471193e+03,4.726813e+03,6.997172e+03,4.875689e+03,37219.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,riverside county ca,22.5,15.8,13.7,12.8,23.6,11.7,2.572823e+05,1.806694e+05,1.566563e+05,1.463651e+05,2.698606e+05,1.337868e+05,1143477.0
2016,riverside county ca,22.3,15.5,13.8,12.7,23.7,12.0,2.578161e+05,1.791995e+05,1.595454e+05,1.468280e+05,2.740019e+05,1.387351e+05,1156126.0
2017,riverside county ca,21.4,14.9,13.7,12.8,23.7,13.5,-1.902222e+08,-1.324444e+08,-1.217778e+08,-1.137778e+08,-2.106667e+08,-1.200000e+08,-888888888.0


In [32]:
final_sorted = final.sort_index()

In [33]:
final_sorted

,location_key,0-14 years perc,15-24 years perc,25-34 years perc,35-44 years perc,45-64 years perc,65+ years perc,0-14 years,15-24 years,25-34 years,35-44 years,45-64 years,65+ years,Total population
year,,,,,,,,,,,,,,
2012,cathedral city ca,20.7,16.6,11.5,12.6,24.7,13.7,5.312034e+03,4.259892e+03,2.951130e+03,3.233412e+03,6.338514e+03,3.515694e+03,25662.0
2012,riverside county ca,23.6,16.1,13.1,13.4,22.9,10.9,2.576941e+05,1.757998e+05,1.430420e+05,1.463178e+05,2.500506e+05,1.190197e+05,1091924.0
2012,rancho mirage ca,7.0,4.2,3.8,8.4,32.9,43.7,6.013000e+02,3.607800e+02,3.264200e+02,7.215600e+02,2.826110e+03,3.753830e+03,8590.0
2012,palm springs ca,8.7,7.3,8.5,12.2,39.4,23.9,2.208408e+03,1.853032e+03,2.157640e+03,3.096848e+03,1.000130e+04,6.066776e+03,25384.0
2012,palm desert ca,13.6,12.0,9.3,8.1,24.2,32.7,3.146904e+03,2.776680e+03,2.151927e+03,1.874259e+03,5.599638e+03,7.566453e+03,23139.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,desert hot springs ca,20.7,13.1,12.9,14.9,24.7,13.5,-1.840000e+08,-1.164444e+08,-1.146667e+08,-1.324444e+08,-2.195556e+08,-1.200000e+08,-888888888.0
2019,coachella ca,19.1,13.8,17.8,15.3,25.7,8.1,-1.697778e+08,-1.226667e+08,-1.582222e+08,-1.360000e+08,-2.284444e+08,-7.200000e+07,-888888888.0
2019,cathedral city ca,18.6,13.0,12.6,12.5,26.2,17.1,-1.653333e+08,-1.155556e+08,-1.120000e+08,-1.111111e+08,-2.328889e+08,-1.520000e+08,-888888888.0


In [34]:
cathedral = final_sorted[final_sorted['location_key'] == 'cathedral city ca']
city_year = cathedral[cathedral.index == int(current_year)]
city_year

,location_key,0-14 years perc,15-24 years perc,25-34 years perc,35-44 years perc,45-64 years perc,65+ years perc,0-14 years,15-24 years,25-34 years,35-44 years,45-64 years,65+ years,Total population
year,,,,,,,,,,,,,,
2016,cathedral city ca,20.4,13.7,13.1,12.9,25.8,14.1,5748.72,3860.66,3691.58,3635.22,7270.44,3973.38,28180.0


In [35]:
city = 'cathedral city ca'
coachella = final_sorted[final_sorted['location_key'] == 'Coachella Valley']
riverside = final_sorted[final_sorted['location_key'] == 'riverside county ca']
city_isolate = final_sorted[final_sorted['location_key'] == city]

riverside

,location_key,0-14 years perc,15-24 years perc,25-34 years perc,35-44 years perc,45-64 years perc,65+ years perc,0-14 years,15-24 years,25-34 years,35-44 years,45-64 years,65+ years,Total population
year,,,,,,,,,,,,,,
2012,riverside county ca,23.6,16.1,13.1,13.4,22.9,10.9,2.576941e+05,1.757998e+05,1.430420e+05,1.463178e+05,2.500506e+05,1.190197e+05,1091924.0
2013,riverside county ca,23.3,16.1,13.3,13.1,23.2,11.1,2.585580e+05,1.786603e+05,1.475889e+05,1.453695e+05,2.574483e+05,1.231757e+05,1109691.0
2014,riverside county ca,22.9,15.9,13.5,13.0,23.5,11.3,2.582946e+05,1.793399e+05,1.522697e+05,1.466301e+05,2.650621e+05,1.274554e+05,1127924.0
2015,riverside county ca,22.5,15.8,13.7,12.8,23.6,11.7,2.572823e+05,1.806694e+05,1.566563e+05,1.463651e+05,2.698606e+05,1.337868e+05,1143477.0
2016,riverside county ca,22.3,15.5,13.8,12.7,23.7,12.0,2.578161e+05,1.791995e+05,1.595454e+05,1.468280e+05,2.740019e+05,1.387351e+05,1156126.0
2017,riverside county ca,21.4,14.9,13.7,12.8,23.7,13.5,-1.902222e+08,-1.324444e+08,-1.217778e+08,-1.137778e+08,-2.106667e+08,-1.200000e+08,-888888888.0
2018,riverside county ca,21.2,14.6,13.8,12.8,24.0,13.8,-1.884444e+08,-1.297778e+08,-1.226667e+08,-1.137778e+08,-2.133333e+08,-1.226667e+08,-888888888.0
2019,riverside county ca,21.0,14.4,13.9,12.7,23.9,14.2,-1.866667e+08,-1.280000e+08,-1.235556e+08,-1.128889e+08,-2.124444e+08,-1.262222e+08,-888888888.0


In [36]:
riverside_year = riverside[riverside.index == int(current_year)]
coachella_year = coachella[coachella.index == int(current_year)]
city_year = city_isolate[city_isolate.index == int(current_year)]

riverside_year

,location_key,0-14 years perc,15-24 years perc,25-34 years perc,35-44 years perc,45-64 years perc,65+ years perc,0-14 years,15-24 years,25-34 years,35-44 years,45-64 years,65+ years,Total population
year,,,,,,,,,,,,,,
2016,riverside county ca,22.3,15.5,13.8,12.7,23.7,12.0,257816.098,179199.53,159545.388,146828.002,274001.862,138735.12,1156126.0


In [37]:
city_year

,location_key,0-14 years perc,15-24 years perc,25-34 years perc,35-44 years perc,45-64 years perc,65+ years perc,0-14 years,15-24 years,25-34 years,35-44 years,45-64 years,65+ years,Total population
year,,,,,,,,,,,,,,
2016,cathedral city ca,20.4,13.7,13.1,12.9,25.8,14.1,5748.72,3860.66,3691.58,3635.22,7270.44,3973.38,28180.0


In [38]:
# city_year
# city_final = city_year.drop(columns=['location_key'])
rcxcity = riverside_year.drop(columns=['location_key']).subtract(city_year.drop(columns=['location_key']))
# rcxcity.info()
bar_1 = 100*(rcxcity.iloc[0]['0-14 years']/rcxcity.iloc[0]['Total population'])
bar_1

22.347468584488976

In [41]:
city = 'cathedral city ca'

coachella = final_sorted[final_sorted['location_key'] == 'Coachella Valley']
riverside = final_sorted[final_sorted['location_key'] == 'riverside county ca']
riverside_year = riverside[riverside.index == int(current_year)]
coachella_year = coachella[coachella.index == int(current_year)]

def generate_age_plot(city: str):
    
    city_isolate = final_sorted[final_sorted['location_key'] == city]
    city_year = city_isolate[city_isolate.index == int(current_year)]
    
    rcxcity = riverside_year.drop(columns=['location_key']).subtract(city_year.drop(columns=['location_key']))
    cvxcity = coachella_year.drop(columns=['location_key']).subtract(city_year.drop(columns=['location_key']))
    
    bar_1 = 100*(rcxcity.iloc[0]['0-14 years']/rcxcity.iloc[0]['Total population'])
    bar_2 = 100*(rcxcity.iloc[0]['15-24 years']/rcxcity.iloc[0]['Total population'])
    bar_4 = 100*(rcxcity.iloc[0]['35-44 years']/rcxcity.iloc[0]['Total population'])
    bar_5 = 100*(rcxcity.iloc[0]['45-64 years']/rcxcity.iloc[0]['Total population'])
    bar_6 = 100*(rcxcity.iloc[0]['65+ years']/rcxcity.iloc[0]['Total population'])
    bar_3 = 100*(rcxcity.iloc[0]['25-34 years']/rcxcity.iloc[0]['Total population'])
    
    bar_1_cv = 100*(cvxcity.iloc[0]['0-14 years']/cvxcity.iloc[0]['Total population'])
    bar_2_cv = 100*(cvxcity.iloc[0]['15-24 years']/cvxcity.iloc[0]['Total population'])
    bar_4_cv = 100*(cvxcity.iloc[0]['35-44 years']/cvxcity.iloc[0]['Total population'])
    bar_5_cv = 100*(cvxcity.iloc[0]['45-64 years']/cvxcity.iloc[0]['Total population'])
    bar_6_cv = 100*(cvxcity.iloc[0]['65+ years']/cvxcity.iloc[0]['Total population'])
    bar_3_cv = 100*(cvxcity.iloc[0]['25-34 years']/cvxcity.iloc[0]['Total population'])
    
    x_axis=[city, 'riverside county ca', 'Coachella Valley']
    
    fig = go.Figure(data=[
        go.Bar(name='0-14 years perc', x=x_axis, y=[city_year.iloc[0]['0-14 years perc'], bar_1, bar_1_cv]),
        go.Bar(name='15-24 years perc', x=x_axis, y=[city_year.iloc[0]['15-24 years perc'], bar_2, bar_2_cv]),
        go.Bar(name='25-34 years perc', x=x_axis, y=[city_year.iloc[0]['15-24 years perc'], bar_3, bar_3_cv]),
        go.Bar(name='35-44 years perc', x=x_axis, y=[city_year.iloc[0]['35-44 years perc'], bar_4, bar_4_cv]),
        go.Bar(name='45-64 years perc', x=x_axis, y=[city_year.iloc[0]['45-64 years perc'], bar_5, bar_5_cv]),
        go.Bar(name='65+ years perc', x=x_axis, y=[city_year.iloc[0]['65+ years perc'], bar_6, bar_6_cv])
    ])
    # Change the bar mode
    fig.update_layout(barmode='group')
    fig.show()

In [42]:
generate_age_plot(city)